# Batch Process multiple freemocap session folders


In [1]:
#import stuff
from pathlib import Path
from typing import Union
print('a')
from src.blender_stuff.create_blend_file_from_session_data import create_blend_file_from_session_data
print('b')
from src.core_processes.capture_volume_calibration.anipose_camera_calibration import freemocap_anipose
print('c')
from src.core_processes.capture_volume_calibration.triangulate_3d_data import triangulate_3d_data
print('d')
from src.core_processes.mediapipe_stuff.convert_mediapipe_npy_to_csv import convert_mediapipe_npy_to_csv
print('e')
from src.core_processes.mediapipe_stuff.mediapipe_skeleton_detector import MediaPipeSkeletonDetector
print('f')
from src.core_processes.post_process_skeleton_data.gap_fill_filter_and_origin_align_skeleton_data import    gap_fill_filter_origin_align_3d_data_and_then_calculate_center_of_mass
print('g')

a
b
c
d
[2022-10-17 11:13:51.0108] [   DEBUG] [matplotlib] [wrapper():277] [PID:40280 TID:35620] matplotlib data path: D:\Dropbox\GitKrakenRepos\freemocap\.venv\lib\site-packages\matplotlib\mpl-data
[2022-10-17 11:13:51.0112] [   DEBUG] [matplotlib] [wrapper():277] [PID:40280 TID:35620] CONFIGDIR=C:\Users\jonma\.matplotlib
[2022-10-17 11:13:51.0114] [   DEBUG] [matplotlib] [<module>():1447] [PID:40280 TID:35620] interactive is False
[2022-10-17 11:13:51.0115] [   DEBUG] [matplotlib] [<module>():1448] [PID:40280 TID:35620] platform is win32
[2022-10-17 11:13:51.0115] [   DEBUG] [matplotlib] [<module>():1449] [PID:40280 TID:35620] loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'nt', 'winreg', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', 'encodings.cp1252', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '

In [2]:
# getcher paths straight
path_to_folder_of_session_folders = Path(
    r'D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data')

path_to_camera_calibration_toml = Path(
    r"D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data\sesh_2022-09-28_15_57_08_calibration\sesh_2022-09-28_15_57_08_calibration_calibration.toml"
)

path_to_blender_executable = Path(r"C:\Program Files\Blender Foundation\Blender 3.2\blender.exe")

In [3]:
# get anipose calibration object
anipose_calibration_object = freemocap_anipose.CameraGroup.load(str(path_to_camera_calibration_toml))

In [4]:
def process_session_folder(synchronized_videos_folder: Union[str, Path],
                           output_data_folder: Union[str, Path],
                           ):
    if not synchronized_videos_folder.exists():
        raise FileNotFoundError(f'Could not find synchronized_videos folder at {synchronized_videos_folder}')

    print("Detecting 2d skeletons...")
    # 2d skeleton detection
    mediapipe_skeleton_detector = MediaPipeSkeletonDetector()
    mediapipe_2d_data = mediapipe_skeleton_detector.process_folder_full_of_videos(
        synchronized_videos_folder, output_data_folder
    )

    print("Triangulating 3d skeletons...")
    # 3d skeleton triangulation
    skel3d_frame_marker_xyz, skeleton_reprojection_error_fr_mar = triangulate_3d_data(
        anipose_calibration_object=anipose_calibration_object,
        mediapipe_2d_data=mediapipe_2d_data,
        output_data_folder_path=output_data_folder,
        mediapipe_confidence_cutoff_threshold=.7,
        use_triangulate_ransac=True,
    )

    print("Gap-filling, butterworth filtering, origin aligning 3d skeletons, then calculating center of mass ...")
    # Post Process raw data
    gap_fill_filter_origin_align_3d_data_and_then_calculate_center_of_mass(
        skel3d_frame_marker_xyz,
        skeleton_reprojection_error_fr_mar=skeleton_reprojection_error_fr_mar,
        data_arrays_path=output_data_folder,
        sampling_rate=30,
        cut_off=10,
        order=4,
        reference_frame_number=None,
    )

    print("Breaking up big `npy` into smaller bits and converting to `csv`...")
    # break up big NPY and save out csv's
    convert_mediapipe_npy_to_csv(
        mediapipe_3d_frame_trackedPoint_xyz=skel3d_frame_marker_xyz,
        output_data_folder_path=output_data_folder,
    )

    print("Creating Blender animation from motion capture data...")
    # export to Blender
    create_blend_file_from_session_data(
        session_folder_path=Path(synchronized_videos_folder).parent, blender_exe_path=path_to_blender_executable
    )


In [5]:
#get paths to folder full of session folders
list_of_session_folders = list(path_to_folder_of_session_folders.glob('ses*'))
print(list_of_session_folders)

[WindowsPath('D:/Dropbox/Northeastern/Courses/Biol2299/2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior/freemocap_bos_com_standing_data/FreeMocap_Data/sesh_2022-09-28_15_39_31_calibration'), WindowsPath('D:/Dropbox/Northeastern/Courses/Biol2299/2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior/freemocap_bos_com_standing_data/FreeMocap_Data/sesh_2022-09-28_15_45_39'), WindowsPath('D:/Dropbox/Northeastern/Courses/Biol2299/2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior/freemocap_bos_com_standing_data/FreeMocap_Data/sesh_2022-09-28_15_52_24_bbbbbb'), WindowsPath('D:/Dropbox/Northeastern/Courses/Biol2299/2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior/freemocap_bos_com_standing_data/FreeMocap_Data/sesh_2022-09-28_15_57_08_calibration'), WindowsPath('D:/Dropbox/Northeastern/Courses/Biol2299/2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior/freemocap_bos_com_standing_data/FreeMocap_Data/sesh_2022-09-28_15_58_27')

In [6]:
session_type = 'pre-alpha'
for session_folder in list_of_session_folders:
    print(f'Processing {session_folder}')
    if session_type == 'pre-alpha':
        synchronized_videos_folder = Path(session_folder) / 'SyncedVideos'
    else:
        synchronized_videos_folder = Path(session_folder) / 'synchronized_videos'

    output_data_folder = Path(session_folder) / 'output_data'
    output_data_folder.mkdir(exist_ok=True, parents=True)
    process_session_folder(synchronized_videos_folder=synchronized_videos_folder,
                           output_data_folder=output_data_folder,)

print("Done!")

Processing D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data\sesh_2022-09-28_15_39_31_calibration
Creating Blender animation from motion capture data...
[2022-10-17 11:13:51.0559] [    INFO] [src.blender_stuff.create_blend_file_from_session_data] [create_blend_file_from_session_data():53] [PID:40280 TID:35620] Starting `blender` sub-process with this command: 
 C:\Program Files\Blender Foundation\Blender 3.2\blender.exe --background --python D:\Dropbox\GitKrakenRepos\freemocap\src\blender_stuff\bind_rigify_meta_rig_to_freemocap_data.py -- D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data\sesh_2022-09-28_15_39_31_calibration 0
Blender 3.2.1 (hash a2d59b2dac9e built 2022-07-06 07:45:50)
Read prefs: C:\Users\jonma\AppData\Roaming\Blender Foundation\Blender\3.2\config\userpref.blend
Erro